In [ ]:
"""
reference
langchain doc: https://python.langchain.com/en/latest/
Use LangChain, GPT and Deep Lake to work with code base: https://python.langchain.com/en/latest/use_cases/code/code-analysis-deeplake.html

"""

SyntaxError: ignored

In [ ]:
# We need to set up keys for external services and install necessary python libraries
!python3 -m pip install --upgrade langchain deeplake openai ticktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import os
from getpass import getpass

# 到openai申請api key (有免費額度)
# https://platform.openai.com/account/api-keys

# Please manually enter OpenAI Key
os.environ['OPENAI_API_KEY'] = getpass('Open API key:')

# Authenticate into Deep Lake if you want to create your own dataset and publish it. 
# You can get an API key from the platform at app.activeloop.ai
os.environ['ACTIVELOOP_TOKEN'] = getpass('Activeloop Token:')

Open API key:··········
Activeloop Token:··········


In [2]:
# Load all repository files. 
# Here we assume this notebook is downloaded as the part of the langchain fork and we work with the python files of the langchain repo.

from langchain.document_loaders import TextLoader

root_dir = '../../../..'

docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith('.py') and '/.venv/' not in dirpath:
            try: 
                loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
                docs.extend(loader.load_and_split())
            except Exception as e: 
                pass
print(f'{len(docs)}')

238821


In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
print(f"{len(texts)}")

串流輸出內容已截斷至最後 5000 行。


Then embed chunks and upload them to the DeepLake.
This can take several minutes.

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(disallowed_special=())
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special=set(), chunk_size=1000, max_retries=6)

In [7]:
!pip install tiktoken
from langchain.vectorstores import DeepLake
import tiktoken

db = DeepLake.from_documents(texts, embeddings, dataset_path=f"hub://francescatai/langchain-code")
db

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/francescatai/langchain-code



|

hub://francescatai/langchain-code loaded successfully.



Dataset(path='hub://francescatai/langchain-code', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype    shape    dtype  compression
  -------   -------  -------  -------  ------- 
 embedding  generic   (0,)    float32   None   
    ids      text     (0,)      str     None   
 metadata    json     (0,)      str     None   
   text      text     (0,)      str     None   


Evaluating ingest: 33%|███▎      | 276/827 [23:05<45:19WARNING:langchain.embeddings.openai:Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Evaluating ingest: 33%|███▎      | 276/827 [23:39<47:14


TransformError: ignored